# Creating a Character Level Language Model

Downloading Dataset

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-30 12:30:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2024-06-30 12:30:56 (15.9 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



Reading dataset

In [4]:
with open('input.txt', 'r', encoding = 'utf-8') as file:
    text = file.read()

In [5]:
print(f"Length of dataset(chars): {len(text)} \n")
print(text[:1000])

Length of dataset(chars): 1115394 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for

Taking unique chars in text

In [6]:
chars = sorted(set(text))
vocabSize = len(chars)
print(''.join(chars))
print(vocabSize)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Mapping from chars to ints

# When redoing, do with google SentencePiece or openAI tiktoken

subword, unit level tokenizer (not individual chars but not full word)

In [7]:
stoi = {ch:i for i, ch in enumerate(chars)}  #dict mapping all chars to their ascii vals
itos = {i:ch for i, ch in enumerate(chars)}  #dict mapping all ascii vals(ints) to their chars
encode = lambda s: [stoi[c] for c in s] #lambda func encoder --> string to list of ints
decode = lambda l: ''.join([itos[i] for i in l]) #decoder --> list of ints to str

In [8]:
print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [9]:
!pip install torch

In [10]:
import torch

data = torch.tensor(encode(text), dtype = torch.long) #makes the basic datatype in pytorch, like a numpy array
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

Split data into training and validation

In [11]:
n = int(.9 * len(data))
trainData = data[:n]
valData = data[n:]

Start of training

In [12]:
blockSize = 8 #maximum length of str to input into model at a time
trainData[:blockSize + 1] #plus one because there

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = trainData[:blockSize] #inputs to the transformer (first block size chars)
y = trainData[:blockSize + 1] #targets for each positions in the inputs
for t in range(blockSize):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

#transformer will never recieve more than blocksize inputs

When input is tensor([18]) the target: 18
When input is tensor([18, 47]) the target: 47
When input is tensor([18, 47, 56]) the target: 56
When input is tensor([18, 47, 56, 57]) the target: 57
When input is tensor([18, 47, 56, 57, 58]) the target: 58
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 47


In [14]:
#batches, process multiple chunks independently at the same time
torch.manual_seed(360)
batchSize = 4    #how many sequences processed in paralel
blockSize = 8    #max context length for predictions

def getBatch(split):
    """
    Function that generates a small batch of 
    data of inputs x and targets y
    """
    if split == 'train':
        data = trainData
    else:
        valData

    ix = torch.randint(len(data) - blockSize, (batchSize,)) #4 randomly generated numbers that are offsets into the training setss
    x = torch.stack([data[i:i + blockSize] for i in ix])
    y = torch.stack([data[i + 1 : i + blockSize + 1] for i in ix])
    return x, y

xb, yb = getBatch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('Targets: ')
print(yb.shape)
print(yb)

print()
print ('--------------------------------')

for b in range(batchSize): # batch dimension
    for t in range(blockSize): # time dimension
        context = xb[b, : t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[46, 43, 56,  1, 44, 53, 56,  1],
        [58, 46, 43,  1, 42, 59, 49, 43],
        [50, 57,  6,  0, 26, 53, 58,  1],
        [53, 56, 49,  8,  0,  0, 19, 24]])
Targets: 
torch.Size([4, 8])
tensor([[43, 56,  1, 44, 53, 56,  1, 43],
        [46, 43,  1, 42, 59, 49, 43,  8],
        [57,  6,  0, 26, 53, 58,  1, 61],
        [56, 49,  8,  0,  0, 19, 24, 27]])

--------------------------------
When input is [46] the target: 43
When input is [46, 43] the target: 56
When input is [46, 43, 56] the target: 1
When input is [46, 43, 56, 1] the target: 44
When input is [46, 43, 56, 1, 44] the target: 53
When input is [46, 43, 56, 1, 44, 53] the target: 56
When input is [46, 43, 56, 1, 44, 53, 56] the target: 1
When input is [46, 43, 56, 1, 44, 53, 56, 1] the target: 43
When input is [58] the target: 46
When input is [58, 46] the target: 43
When input is [58, 46, 43] the target: 1
When input is [58, 46, 43, 1] the target: 42
When input is [58, 46, 43, 1, 42] the 

In [15]:
print(xb) # our inpit to the transformer

tensor([[46, 43, 56,  1, 44, 53, 56,  1],
        [58, 46, 43,  1, 42, 59, 49, 43],
        [50, 57,  6,  0, 26, 53, 58,  1],
        [53, 56, 49,  8,  0,  0, 19, 24]])


In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(360)

"""
B: Batch size. This is the number of sequences processed in parallel during training or inference. The number of sequences in a single batch.

T: Time steps. This is the length of each sequence or the number of tokens in each sequence. Often corresponds to the number of words or characters in a sequence.

C: Channel size. This is the size of the embedding for each token in the sequence. Csorresponds to the vocabulary size because the embeddings are of size vocabSize.
"""

class BigramLanguageModel(nn.Module): #simplest language model

    def __init__(self, vocabSize):
        super().__init__()
        #token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocabSize, vocabSize)

    def forward(self, idx, targets=None):
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(Batch, Time, Channel (vocabSize))
        #logits = scores for next char in the series

        #transform logits, targets shape to better conform to what pytorch expects 
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # - log likelihood loss (cross-entropy)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    #history is not used right now, but will be used later
    def generate(self, idx, maxNewTokens):
        #idx is (B, T) array of indices in curr context
        for _ in range(maxNewTokens):
            logits, loss = self(idx) # get the new preds (logits)
            logits = logits[:, -1, :] #transforms from B, C to B, T, C
            probs = F.softmax(logits, dim=-1) #apply softmax to get probs
            idxNext = torch.multinomial(probs, num_samples = 1) # sample from the distr (B, 1)
            idx = torch.cat((idx, idxNext), dim=1) #append samled index to running seq(B, T + 1)
        return idx

m = BigramLanguageModel(vocabSize)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype = torch.long) #creating  a tensore (batch = 1x time = 1), initialized with 0
print(decode(m.generate(idx, maxNewTokens=100)[0].tolist())) # asking for 100 tokens

torch.Size([32, 65])
tensor(4.6558, grad_fn=<NllLossBackward0>)

DU'i$VEXYXFgS;xWPxFrePG;$Erj3HxEQRgtCl

QujbwQUuBW bGKXJN$IDAJ lPd$EJFbQFfkT:;AQEadXR:UuJSeKwQyQIDBR


Output is really bad because this a totally random model, now train this models

In [17]:
#create a pytorch optimzer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #AdamW as opposed to SGD, more advanced and popular

In [18]:
batchSize = 32
for steps in range(10000):

    #sample a batch of data
    xb, yb = getBatch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3918263912200928


In [19]:
idx = torch.zeros((1, 1), dtype = torch.long) #creating  a tensore (batch = 1x time = 1), initialized with 0
print(decode(m.generate(idx, maxNewTokens=400)[0].tolist())) # asking for 100 tokens



And ngmio O:
MENard on, d nt oriver brre thouriinde thomepldos g s llang our thond te
Fisthy.
HENTUCA:
MNIIOUSThilai-alyond al'dot ferafaind he, r an as! thelll!
INGS:

Hor, pet fig y ls wr s,
has bl EDE:
s ises:

f keh sck howitiders LOfat thisewive heshanar llincark intoun, The ireshe chepus
TRCKllle mu; de, gaus then,
Card nexd olllil thanowfaput sthe, nghyrmy evess, athakeveady d 'd
MEOnde at


The mathematical trick in self-attention

In [21]:
#toy examples

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

Want to couple tokens with those behind it
- For every t_th token, average the vectors of those behind it

In [24]:
xbow = torch.zeros((B,T,C)) # x bag of words (used for averaging)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b, t] = torch.mean(xprev, 0)

In [35]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [38]:
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

Making weighted aggregations of past elementss

In [ ]:
#usign softmax
tril = torch.tril(torch.ones(T, T)) #weights, but all weights are 1/0
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) #making sure tokens from the past can't communitcate
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x #(B, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow3)

In [ ]:
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))

In [30]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [25]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [26]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [27]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(c)

#the lower triangular matrix of all 1s muiltiplied by b gives a matrix w sums

tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [45]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

#single head performing self-attention
headSize = 16
key = nn.Linear(C, headSize, bias=False)
query = nn.Linear(C, headSize, bias=False)
value = nn.Linear(C, headSize, bias=False)
k = key(x) # (B, T, 16(headSize))
q = query(x) #(B, T, 16(headSize))

#no communication has happened yet, now time for communication
wei = q @ k.transpose(-2, -1) # (B, T, 16) 

tril = torch.tril(torch.ones(T, T)) #weights, but all weights are 1/0
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) #making sure tokens from the past can't communitcate
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v #(B, T, T) @ (B, T, C) ---> (B, T, 16) @ (B, 16, T) --> (B, T, T)

out.shape

torch.Size([4, 8, 16])

In [43]:
wei

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [5.5992e-01, 4.4008e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [3.2197e-01, 2.0162e-01, 4.7641e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.6397e-01, 8.1458e-02, 2.9607e-01, 4.5850e-01, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.0508e-01, 3.0070e-01, 1.8936e-01, 1.8076e-01, 1.2409e-01,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.0019e-02, 1.2732e-01, 2.9082e-02, 1.6934e-02, 5.5211e-02,
          7.1144e-01, 0.0000e+00, 0.0000e+00],
         [1.4085e-01, 1.0251e-01, 1.7444e-01, 2.0379e-01, 1.6895e-01,
          6.6888e-02, 1.4256e-01, 0.0000e+00],
         [2.2274e-02, 1.0857e-01, 8.2257e-03, 4.0053e-03, 8.0379e-03,
          7.2572e-01, 2.1608e-02, 1.0156e-01]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00